In [1]:
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm

In [6]:
members = pd.read_excel(Path.cwd().parent / 'data/msci_acwi' / 'mxwd_members.xlsx')

In [4]:
meta = pd.read_excel(Path.cwd().parent / 'data/msci_acwi' / 'mxwd_members_unique.xlsx')

In [10]:
meta = pd.read_excel(Path.cwd().parent / 'data' / 'msci_acwi' / 'mxwd_members_unique.xlsx')
meta.columns = [c.lower() for c in meta.columns]
meta.drop(['weight', 'country_full_name', 'company_name', 'index_date'], axis=1, inplace=True)
meta.dropna(how='any', inplace=True)
meta.columns = ['ticker', 'gics_sector', 'gics_industry', 'gics_industry_group', '_code', 'country']
meta = meta[meta.country.isin(['US', 'CH', 'JN', 'SK'])]

In [11]:
meta

,ticker,gics_sector,gics_industry,gics_industry_group,_code,country
0,4091 JT Equity,Materials,Chemicals,Materials,JP3711600001,JN
2,8960 JT Equity,Real Estate,Diversified REITs,Equity Real Estate Investment,JP3045540006,JN
3,9876544D UN Equity,Consumer Discretionary,"Hotels, Restaurants & Leisure",Consumer Services,US4599021023,US
6,023530 KP Equity,Consumer Discretionary,Broadline Retail,Consumer Discretionary Distrib,KR7023530009,SK
12,7262 JT Equity,Consumer Discretionary,Automobiles,Automobiles & Components,JP3496600002,JN
...,...,...,...,...,...,...
4844,WSM UN Equity,Consumer Discretionary,Specialty Retail,Consumer Discretionary Distrib,US9699041011,US
4845,LNG UN Equity,Energy,"Oil, Gas & Consumable Fuels",Energy,US16411R2085,US
4847,IOT UN Equity,Information Technology,Software,Software & Services,US79589L1061,US
4848,CRBG UN Equity,Financials,Financial Services,Financial Services,US21871X1090,US


In [49]:
df = pd.merge(members, meta, how='inner', on='ticker').sort_values(['index_date', '_code']).reset_index(drop=True)

In [50]:
df['index_date'] = pd.to_datetime(df['index_date'], format="%Y-%m-%d").dt.strftime('%Y-%m-01')

In [59]:
result = pd.merge(
    df.drop_duplicates('_code', keep='last').reset_index(drop=True),
    df.drop_duplicates('_code', keep='first').reset_index(drop=True)[['_code', 'index_date']],
    how='left',
    on='_code'
)

In [60]:
result = result[['_code', 'ticker', 'company_name', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry', 'index_date_x', 'index_date_y']]
result.columns = ['_code', 'ticker', 'company_name', 'country', 'gics_sector', 'gics_industry_group', 'gics_industry', 'last_include', 'first_include']

In [61]:
result['last_include'] = (pd.to_datetime(result['last_include'], format="%Y-%m-%d") + pd.DateOffset(months=3, days=-1)).dt.strftime('%Y-%m-%d')

In [64]:
result.sort_values(['country', 'company_name']).to_csv(Path.cwd().parent / 'data/msci_acwi' / 'meta.csv', index=False)

: 